## Importing necessary libraries

In [22]:
import pandas as pd
import re
import inflect
import numpy as np
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [100]:
# Read data into pandas DataFrames
reviews = pd.read_csv('./train_labelled.csv', encoding='unicode_escape')

In [101]:
reviews.isnull().any()

Restaurant    False
Review        False
Label         False
Stars         False
Date          False
dtype: bool

In [102]:
reviews.shape

(1922, 5)

In [103]:
print(X[1])

small selection italian wine glass beer well good wine list


In [104]:
reviews.head()

,Restaurant,Review,Label,Stars,Date
0,Happy Tummy,"Fresh ingredients, friendly peeps and so much ...",1,5,2016-04-06T00:00:00
1,Cibo Italiano,A small selection of Italian wines by the glas...,1,4,2015-12-24T00:00:00
2,Yan kee Noodle House,The plus point is that the price remains the s...,1,4,2018-12-28T00:00:00
3,Clinton Street Baking Company & Restaurant,Same for more?I ordered what I thought was the...,1,2,2018-03-03T00:00:00
4,Song Fa Bak Kut Teh,I will probably get that again.Have been very ...,1,5,2019-01-05T00:00:00


## Data Preprocessing

In [112]:
# Cleaning the Data
def clean(data):

    # Removing leading and trailing white spaces
    data = str(data).strip()
    
    # Converting all text to lower case
    data = data.lower() 
    
    # add space for punctuation
    translator = re.compile('[%s]' % re.escape(string.punctuation))
    translator.sub(' ', data)

    #remove punct
    data = re.sub('[^A-Za-z0-9]+', ' ', data)
    
    #keep english words
    data = re.sub(r'[^\x00-\x7F]+', '', data)
    
    #keep printable
    data = re.sub(f'[^{re.escape(string.printable)}]', '', data)

    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(data)

    # Converting all text to base form
#     lemmatizer = WordNetLemmatizer()
#     tokens_lemmatized = [lemmatizer.lemmatize(token) for token in tokens]
            
    # Removing Punctuation
#     translator = str.maketrans('', '', string.punctuation)
#     data = data.translate(translator)

         # Converting all numbers to words
#    word = data.split(' ')
#     p = inflect.engine()
#     word = ' '.join([i for i in word if not i.isdigit()])
#     word = re.sub(r'\d+', p.number_to_words(word) , word)    

    word = ' '.join([i for i in tokens if not i in stop_words])

    return word

# clean text data
reviews["Review_clean"] = reviews["Review"].apply(lambda x: clean(x))
reviews

,Restaurant,Review,Label,Stars,Date,Review_clean
0,Happy Tummy,"Fresh ingredients, friendly peeps and so much ...",1,5,2016-04-06T00:00:00,fresh ingredients friendly peeps much cheaper ...
1,Cibo Italiano,A small selection of Italian wines by the glas...,1,4,2015-12-24T00:00:00,small selection italian wines glass beers well...
2,Yan kee Noodle House,The plus point is that the price remains the s...,1,4,2018-12-28T00:00:00,plus point price remains 4 per bowl
3,Clinton Street Baking Company & Restaurant,Same for more?I ordered what I thought was the...,1,2,2018-03-03T00:00:00,ordered thought signature omelette name
4,Song Fa Bak Kut Teh,I will probably get that again.Have been very ...,1,5,2019-01-05T00:00:00,probably get tempted pick soup base sell sure ...
...,...,...,...,...,...,...
1917,Neko no Niwa,All the kitties are super soft and I really li...,4,4,2015-04-17T00:00:00,kitties super soft really liked art walls deta...
1918,Pu3,The restaurant lacks decorations but the place...,4,5,2016-07-31T00:00:00,restaurant lacks decorations place bright air ...
1919,OverEasy,Get a table (there are couches or high tables)...,4,4,2012-11-15T00:00:00,get table couches high tables outdoors sip coc...
1920,The Naked Finn,The space itself is great.,4,5,2015-02-28T00:00:00,space great


In [113]:
# To get restaurant name and respective reviews
restaurant_names = []
for rest in reviews["Restaurant"]: # each restaurant name
    if rest not in restaurant_names: 
        restaurant_names.append(rest)

reviewsByRestaurant = dict.fromkeys(restaurant_names, "")
for rest in restaurant_names: 
    filtered= []
    reviewsByRestaurant[rest] = {} # create a dictionary for sentences (key = label)
    labels = []
    for label in reviews['Label'][reviews['Restaurant'] == rest]:
        if label not in labels:
            labels.append(label)
#     for cat in labels: 
    for i in range(1, 6):
        reviewsByRestaurant[rest][i] = '' # to compile sentences for a label in a restaurant
        for j in labels:
            if i == j:
                filtered= reviews['Review_clean'][(reviews['Restaurant'] == rest) & (reviews['Label'] == j)] # obtain reviews for a label in a restaurant
                for rev in filtered:
                    reviewsByRestaurant[rest][j] += rev

reviewsByRestaurant

{'Happy Tummy': {1: 'fresh ingredients friendly peeps much cheaper salad stop single gets base romaine 5 toppings 6 80',
  2: '',
  3: '',
  4: '',
  5: ''},
 'Cibo Italiano': {1: 'small selection italian wines glass beers well good wine listgenerous clams',
  2: '',
  3: '',
  4: 'cultural relevant singaporean cuisine sweet old conservation bldg',
  5: ''},
 'Yan kee Noodle House': {1: 'plus point price remains 4 per bowl',
  2: '',
  3: '',
  4: '',
  5: ''},
 'Clinton Street Baking Company & Restaurant': {1: 'ordered thought signature omelette namelooked awful 18 happyexcellent portions massive certainly go hungry',
  2: 'component cooked spinach creamy hollandaise sauce rich crab cake plain deliciousjalapeo cornbread quite large yummycoffee good thoblueberry banana walnut chocolate chunk goodsyrup rich flavorful nuts candies bananas lightly dusted sugar',
  3: 'service prompt guess true test waitstaff peak hours obviously get see would return pancakesseated one staff member came ta

In [114]:
reviews_by_restaurant = pd.DataFrame.from_dict(reviewsByRestaurant)
reviews_by_restaurant = reviews_by_restaurant.transpose()
reviews_by_restaurant = reviews_by_restaurant.reset_index()
# reviews_by_restaurant = reviews_by_restaurant.rename(columns={"index": "Restaurant"})
reviews_by_restaurant.columns =['Restaurant', 'Value', 'Taste', 'Service', 'Ambience', 'Others'] 

reviews_by_restaurant

,Restaurant,Value,Taste,Service,Ambience,Others
0,Happy Tummy,fresh ingredients friendly peeps much cheaper ...,,,,
1,Cibo Italiano,small selection italian wines glass beers well...,,,cultural relevant singaporean cuisine sweet ol...,
2,Yan kee Noodle House,plus point price remains 4 per bowl,,,,
3,Clinton Street Baking Company & Restaurant,ordered thought signature omelette namelooked ...,component cooked spinach creamy hollandaise sa...,service prompt guess true test waitstaff peak ...,,place staplewould go back
4,Song Fa Bak Kut Teh,probably get tempted pick soup base sell sure ...,ordered vegetables table bowl rice side dishes...,go hour waitqueue get pretty long best go earl...,located bukit timah saddle club take chance wa...,never heard asiareflecting past week gluttony ...
...,...,...,...,...,...,...
497,Hopheads,,,super rude disrespectful,,
498,Paulaner BrÃÂ¤uhaus Singapore,,,,good group fun place hang,
499,Twenty Eight Cafe,,,,however want cool cafe nice atmosphere might n...,
500,Brasserie Gavroche,,,,beautiful little place nice atmosphere,


## Classification

In [109]:
X = reviews.Review_clean # feature matrix
y = reviews.Stars # response vector

X.head()

0    fresh ingredient friendly peep much cheaper sa...
1    small selection italian wine glass beer well g...
2                  plus point price remains 4 per bowl
3           ordered thought wa signature omelette name
4    probably get tempted pick soup base sell sure ...
Name: Review_clean, dtype: object

In [107]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [55]:
# Define a function that accepts a vectorizer, a model and calculates the accuracy
def tokenize_predict(vect, model):
    
    # create document-term matrices using the vectorizer
    x_train_dtm = vect.fit_transform(x_train)
    x_test_dtm = vect.transform(x_test)
    
    # print the number of features that were generated
    print('Features: ', x_train_dtm.shape[1])
    
    model.fit(x_train_dtm, y_train)
    
    trainpred = model.predict(x_train_dtm) # for train data
    testpred = model.predict(x_test_dtm) # for test data
    
    # print the accuracy of predictions
    print('Train Accuracy: ', metrics.accuracy_score(y_train, trainpred))
    print('Test Accuracy: ', metrics.accuracy_score(y_test, testpred))
    print('Train AUROC: ', metrics.roc_auc_score(y_train, trainpred))
    print('Test AUROC: ', metrics.roc_auc_score(y_test, testpred))

In [ ]:
# Lemmatize words
class LemmaTokenizer:
     def __init__(self):
         self.wnl = WordNetLemmatizer()

     def __call__(self, doc):
         return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [ ]:
# Import and instantiate Count Vectorizer
vect = CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                , encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 3), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=LemmaTokenizer(),
                vocabulary=None)

In [ ]:
logreg = LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
# (setting multi_class=”ovr”)
# (setting multi_class="multinomial")

In [25]:
dtm = vect.fit_transform(reviews_by_restaurant['compiled'])
dtm.shape
dtm

<770x1060294 sparse matrix of type '<class 'numpy.int64'>'
	with 1515283 stored elements in Compressed Sparse Row format>

In [26]:
features = vect.get_feature_names()
features

['00s',
 '00s emo',
 '00s emo musicyoure',
 '00s rap',
 '00s rap hip',
 '010',
 '010 dont',
 '010 dont want',
 '01012013',
 '01012013 sign',
 '01012013 sign pasted',
 '01102',
 '01102 closed',
 '01102 closed mondays45',
 '0112',
 '0112 2nd',
 '0112 2nd row',
 '01121',
 '01121 offer',
 '01121 offer roasted',
 '01122charsiew',
 '01122charsiew chicken',
 '01122charsiew chicken rice',
 '01127',
 '01127 eat',
 '01127 eat anymore',
 '01147',
 '01147 steamboat',
 '01147 steamboat crab',
 '0132',
 '0132 65',
 '0132 65 6440',
 '0132 lower',
 '0132 lower levelopening',
 '0135this',
 '0135this place',
 '0135this place fried',
 '013972603',
 '013972603 calendar',
 '013972603 calendar years',
 '0156',
 '0156 english',
 '0156 english chinese',
 '0161my',
 '0161my sister',
 '0161my sister thoroughly',
 '0163',
 '0163 3rd',
 '0163 3rd row',
 '0166',
 '0166 cereal',
 '0166 cereal prawns',
 '0183',
 '0183 shoppes',
 '0183 shoppes marina',
 '0185',
 '0185 open',
 '0185 open evenings',
 '0186',
 '0186 off

## Getting TFIDF scores

In [31]:
# To get frequent words using TFIDF

word_scores = {}

def summarize(review_id): # works for each row

    # choose a random review that is at least 400 characters - keep looping until review has > 400 words 
#     review_length = 0
#     while review_length < 400:
#         review_id = np.random.randint(0, data.shape[0])
    review_text = reviews_by_restaurant.loc[review_id, 'compiled']
#         review_length = len(review_text)
    
    # create a list of all unique words in the review (minus stop words) using CountVectorizer
    vect = CountVectorizer(stop_words='english')
    vect.fit([review_text])
    unique_words = vect.get_feature_names()

    # create a dictionary of words and their TF-IDF scores
#     word_scores = {}
    for word in unique_words:
        word_scores[word] = dtm[review_id, features.index(word)]
    
    # print words with the top 5 TF-IDF scores
    print('TOP TF-IDF SCORING WORDS:')
    top_scores = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)[0:5]
    for word, score in top_scores:
        print(word)
    
    # print 5 random words (for comparison)
    print('\n' + 'RANDOM WORDS:')
    random_words = np.random.choice(list(word_scores.keys()), size=5, replace=False)
    for word in random_words:
        print(word)
    
    # print the review
    print('\n' + review_text)

In [32]:
for i in range(len(reviews_by_restaurant)): 
    summarize(i)

TOP TF-IDF SCORING WORDS:
laksa
bowl
noodles
cockles
small

RANDOM WORDS:
ingredientsworth
non
colors
eggs
shrimp

food phenomenal laksa awesome cheap yet heavygot hawker center immediately saw sungei road laksa line formed small wait got bowl delicious laksathe soup dish delicious nice coconut flavor noodles also smooth protein went well dishoverall good bowl laksa price pay would eat im singaporelaksa popular southeast asia spicy noodle soup rice noodle coconut milk curry soup im big fan curry try good definitely worth try handle heat even though dont like curry still enjoyed sungei road seems best also tried laksa colony buffet ritz carlton thought quality pretty similar laksa fraction price 3 small bowllaska thing trying still thing love noodles soup prefer simpler clear broth although im fan soup see popular flavor profile curry coconut chili â bowls small good size youre americas think small chicken bowl size bowl soboro noodles cut utensils available spoons fits perfectly every 

TOP TF-IDF SCORING WORDS:
laksa
crab
cockles
soup
bowls

RANDOM WORDS:
jalan
precut
dedicated
pick
explain

first seafood meal favorite äyã­ family â loves taking love eat perfect â family always makes reservations since several uswe always get 2 large chilli crabs eveyone â absolutely delicious spicy â cant spice refrain trying missing â also got fried rice clams vegetables coconut drinks kids beers adults 1 large white pepper crabservice phenomenal always excellent food â another one favoritesintroduced place friend lives singapore less known location commercial jumbo seafood others get local feel service great explain items make sure know large crab know made big dent final tab ordered white pepper crab chili crab first time white pepper crab enjoyed chili crab chili crab mala taste chili also liked dipping mantou white pepper crab also got clams veggies tasty done well offer gloves utensils get coconut drink crabs came around 15kg check 300 one crab plenty 3 people even 4 wentdesi

TOP TF-IDF SCORING WORDS:
soup
pork
teh
kut
bak

RANDOM WORDS:
dad
ambiance
hahano
discovered
albeit

thanks yelp directing restaurant first meal singapore everi wasnt convinced place warranted reviews saw couple patsy white ribs clearish broth could anything delicious looked boiled seasonings nope nope nope stand corrected admit wrong meat unlike ribs ever broth extra bonus something unassuming simple good pretend even know socks completely knocked soup make grown woman cry yes yes ambiance service simple price low product amazing fly back singapore ribs soup mightbest food ive singapore service fast food came almost immediately bak kut teh die traditional singaporean dish consists braised pork ribs tea broth broth rich rich garlic flavor meat practically falls bone large portion around 10 singaporean dollars 722 us dollars plenty seating get cramped since theres many customers however miss food ton would sure come back ever singaporebest bak kut teh porkrib soup singapore â many good

TOP TF-IDF SCORING WORDS:
crab
chili
good
food
sauce

RANDOM WORDS:
kids
google
freshly
itemized
accommodating

three things buds red dot said must try â tian tian hainanese chicken rice bah kuh teh â singapore chili crab â crossed first two various hawker centers around town â chili crab many recommend come â arrived via metro around 1300 â restaurant half capacity â lunch rush crowd gone reservation needed â yesthe food appetizer shrimp pumpkin purã©e â really good â shrimp fried rice another great dish â enough 2 peoplemain singapore chili crab â excellent â flavorful â spice level right â enough kick enjoy â generous portion good last biteenvironmentservice great â waiter attentive â also let know ordered enough 2 people â glad stopped order two dishes â love bibs large hand washing stations â dont luxury meal worth price â glad listened buds pointed wayunfortunately rating isnt food never got chance try anything never felt unwanted disrespected life took solo trip singapore really

TOP TF-IDF SCORING WORDS:
pepper
kut
bak
seafood
jumbo

RANDOM WORDS:
chaser
lot
weekend
normally
starsafter

dining real treat â enjoyed venue much â located inside large contemporary museum â often get walk thru closed museum quiet intimate bustling oh charming restaurant like â step back time history lining walls framed photos culture embodied gorgeous chandeliers artworks â classy night dress bit feel part ambiance â service friendliest weve anywhere â host made excellent suggestions enjoyed every dish â reservations normally requisite though got lucky arriving without party finished early graciously accommodated usthink miss hit miss situation work holiday party weeks ago learned colleague next dinner used live singapore told eat order buah keluak ayam would change life made reservations dinner last nightwhen mentioned people going lot hype someone said chef national treasure concierge hotel said able get restaurant local surprised able secure reservation anticipation really high 

...

TOP TF-IDF SCORING WORDS:
murtabak
chor
bak
song
fa

RANDOM WORDS:
eveningsgreat
means
refuse
emerald
famoussaw

naked finnwell known seafood dinner tried many times always good especially cold vermicelli prawns tiny squids lunch first time limited menu seafood local foodlobster roll good lobster overcookednaked finn bar called nekkid nearbykeep seeing hairdresser posting yummy food finally last saturday night good foodnice ambience5 us shared one set one set b enough foodliked kang kong loved crispy squid bee hoon scallop carpacciobarramundi fish goodprawn sauce good great although waiter seemed think greatrecently traveled singapore first time heard leading trip clean country excellent food rang true experience found naked fin yelp looking restaurants around love pair food good glass wine images provided yelp page immediately restaurants dont offer drink list small popup shops outdoors humidity bit change socal weather haha thats naked fin stood video quick little blurb owner naked f

TOP TF-IDF SCORING WORDS:
chicken
murtabak
rice
chor
bak

RANDOM WORDS:
moreno
kahali
refillablemall
flow
lucked

thanks yelp helped find favorite hainanese chicken rice place world far got quarter chicken rice set 1580 confuses since thats actually quite expensive hear people say 27 reviews go something wrong price aside first bite rice looked like plain white rice btw chickeny rice heaven chicken perfectly juicy soup good im sold almost always trouble finishing rice places gobbled every last bit meal tried tian tian ah tai still rate highest terms overall taste set obviously 16 twice expensive tian tians chicken juicy sauce rice wasnt comparabledelish inexpensive good fast food 2 good hainesese chicken rice would recommend anyone wants get world hcr trying locals everyone tastes version lighter compared others market oh warned spicy chili sauce super spicy complements well ginger sauce chicken oh yumsimply delicious fast servicechicken rice full flavor08 sgd rice essential go chicken

TOP TF-IDF SCORING WORDS:
murtabak
bak
song
fa
ï¼

RANDOM WORDS:
deliciousâ
feels
annoyingfirst
unimpressive
salads

really best cantonese food around super authentic mouthwateringly delicious incredibly reasonable dont know could possibly ask coming years often crave im back la sure singapore full delicious chinese food food hub however every dish cooked perfection thats sets hong kong street apart cantonese restaurants opinionthere many yummy dishes go phases favorites current favorite pa ting yu steamed silver catfish ask mine black bean sauce firstly fish steamed perfection moist meltinyourmouth goodness secondly sauce point tend steer away black bean sauces salty usually however one anything tell make sauces least thats tastes like something bottle bursting flavor yet salty another dish like sum lor hor fun sliced fish hor fun arent familiar stir fried flat rice noodles bean sprouts sliced fish gravy soupy dry noodles either isnt however black singaporean hor fun might used like o

TOP TF-IDF SCORING WORDS:
murtabak
bak
song
fa
ï¼

RANDOM WORDS:
palatekeep
pursuit
paneer
order6th
hahaknow

much much much better jewel branch nearby official launch last friday course drop see curiosity expecting long queue surprise wasnt around 9ish overenthusiastic staff cheering screaming giving high fives walking thought buy iphone split second lol queueing order breeze 4 counters least 30 staff kitchen churning burgers fries concretes milkshakes efficiently impressive wait manageable around 20 mins order also staff dining floor assisting customers queries passing cutleries interacting kids basically great ambassador brand dining area big spacious enough lots seating booths tall tables plants onto burgers tried various burgers shake shack decided try chicknshack delicious chicken breast slowcooked buttermilk marinate fried perfection served lettuce pickles buttermilk mayo good stuff also tried concretes term used frozen custard blended mixins shack attack regular cantgowrong cho

KeyboardInterrupt: 

In [26]:
word_scores

{'10pm': 1,
 '15': 1,
 'ambiance': 1,
 'arrived': 1,
 'ate': 1,
 'bank': 1,
 'bar': 1,
 'bartenders': 2,
 'basil': 1,
 'bc': 1,
 'blinded': 1,
 'bourbon': 1,
 'break': 1,
 'brought': 1,
 'catch': 1,
 'chill': 1,
 'cocktail': 1,
 'delicious': 1,
 'didnt': 1,
 'disappointed': 1,
 'dont': 1,
 'evening': 1,
 'excellent': 1,
 'experience': 1,
 'fashioned': 1,
 'finely': 1,
 'friday': 2,
 'got': 4,
 'great': 1,
 'hunger': 1,
 'informative': 1,
 'kitchen': 1,
 'lovely': 1,
 'menu': 1,
 'minced': 1,
 'minutes': 1,
 'new': 2,
 'night': 1,
 'oil': 1,
 'old': 1,
 'olive': 1,
 'olives': 1,
 'open': 1,
 'owner': 1,
 'passionate': 1,
 'phenomenal': 1,
 'piggypower': 1,
 'pizza': 2,
 'place': 2,
 'prosciutto': 1,
 'relaxing': 1,
 'remember': 1,
 'right': 1,
 'served': 1,
 'shared': 1,
 'special': 1,
 'spicy': 1,
 'start': 1,
 'theyd': 1,
 'thing': 1,
 'told': 1,
 'warm': 1,
 'wines': 1,
 'work': 1,
 'yummy': 1}